In [73]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import mysql.connector as con
from datetime import datetime
import re

In [2]:
db= con.connect(host='localhost', user='root', password='@Amir123', database='college')
cursor= db.cursor()

## Adding URLs into NEWS_SOURCE

In [36]:
pincode= {'gurgaon':122005,'ghaziabad':201005,'noida':201301,'meerut':250001,'agra':282001,'chandigarh':160017,'kerala':382220,
         'bangalore':560001,'chennai':600001,'coimbatore':641001,'kochi':682001,'malappuram':676505}
port_id= {'gurgaon':122,'ghaziabad':201,'noida':2013,'meerut':2500,'agra':282,'chandigarh':1600,'kerala':3822,
         'bangalore':560,'chennai':600,'coimbatore':641,'kochi':682,'malappuram':6765}
ndtv_urls=[
           "https://www.ndtv.com/gurgaon-news",
           "https://www.ndtv.com/ghaziabad-news",
           "https://www.ndtv.com/noida-news",
           "https://www.ndtv.com/meerut-news",
           "https://www.ndtv.com/agra-news",
           "https://www.ndtv.com/chandigarh-news",
           "https://www.ndtv.com/kerala-news"
           ]
#news_id=1
PORTAL_ID=[]
for i in ndtv_urls:
    for k,v in port_id.items():
        po=re.compile(k)
        if po.search(i):
            PORTAL_ID.append(v)
VENDOR_ID=743
location=[]
for i in ndtv_urls:
    for k,v in pincode.items():
        lo=re.compile(k)
        if lo.search(i):
            location.append(v)
media_name= 'NDTV'

ndtv_records=[]
for i in range(len(ndtv_urls)):
    a=(PORTAL_ID[i],VENDOR_ID,location[i],media_name,ndtv_urls[i])
    ndtv_records.append(a)
a= pd.DataFrame(ndtv_records)
ndtv_records= a.values.tolist()

for i in ndtv_records:
    ndtv_insertion= "INSERT INTO NEWS_SOURCE(PORTAL_ID,VENDOR_ID,LOCATION,MEDIA_NAME,NEWS_URL,TRAINEE_ID) VALUES (%s,%s,%s,%s,%s,743)"
    cursor.execute(ndtv_insertion, i)
    db.commit()
    
th_urls=[
         "https://www.thehindu.com/news/cities/bangalore/",
         "https://www.thehindu.com/news/cities/chennai/",
         "https://www.thehindu.com/news/cities/coimbatore/",
         "https://www.thehindu.com/news/cities/kochi/"
        ]
PORTAL_ID=[]
for i in ndtv_urls:
    for k,v in port_id.items():
        po=re.compile(k)
        if po.search(i):
            PORTAL_ID.append(v)
VENDOR_ID=743
location=[]
for i in th_urls:
    for k,v in pincode.items():
        lo=re.compile(k)
        if lo.search(i):
              location.append(v)
media_name= 'The_Hindu'

th_records=[]
for i in range(len(th_urls)):
    a=(PORTAL_ID[i],VENDOR_ID,location[i],media_name,th_urls[i])
    th_records.append(a)
a= pd.DataFrame(th_records)
th_records= a.values.tolist()
for i in th_records:
    th_insertion= "INSERT INTO NEWS_SOURCE(PORTAL_ID,VENDOR_ID,LOCATION,MEDIA_NAME,NEWS_URL,TRAINEE_ID) VALUES (%s,%s,%s,%s,%s,743)"
    cursor.execute(th_insertion, i)
    db.commit()
    
mlp_urls= ["https://timesofindia.indiatimes.com/topic/malappuram/",
          "https://timesofindia.indiatimes.com/topic/malappuram/2",
          "https://timesofindia.indiatimes.com/topic/malappuram/3"]
PORTAL_ID=6765
VENDOR_ID=743
location=676505
media_name= 'TOI'

mlp_records=[]
for i in range(len(mlp_urls)):
    a=(PORTAL_ID,VENDOR_ID,location,media_name,mlp_urls[i])
    mlp_records.append(a)
a= pd.DataFrame(mlp_records)
mlp_records= a.values.tolist()

for i in mlp_records:
    mlp_insertion= "INSERT INTO NEWS_SOURCE(PORTAL_ID,VENDOR_ID,LOCATION,MEDIA_NAME,NEWS_URL,TRAINEE_ID) VALUES (%s,%s,%s,%s,%s,743)"
    cursor.execute(mlp_insertion, i)
    db.commit()

## Fetching URLs from NEWS_SOURCE

In [25]:
fetching_news_data= "SELECT * FROM NEWS_SOURCE WHERE VENDOR_ID=743"
cursor.execute(fetching_news_data)
news_data= cursor.fetchall()

In [26]:
news_data

[(None,
  122,
  743,
  122005,
  'NDTV',
  'https://www.ndtv.com/gurgaon-news',
  743,
  None),
 (None,
  201,
  743,
  201005,
  'NDTV',
  'https://www.ndtv.com/ghaziabad-news',
  743,
  None),
 (None,
  2013,
  743,
  201301,
  'NDTV',
  'https://www.ndtv.com/noida-news',
  743,
  None),
 (None,
  2500,
  743,
  250001,
  'NDTV',
  'https://www.ndtv.com/meerut-news',
  743,
  None),
 (None, 282, 743, 282001, 'NDTV', 'https://www.ndtv.com/agra-news', 743, None),
 (None,
  1600,
  743,
  160017,
  'NDTV',
  'https://www.ndtv.com/chandigarh-news',
  743,
  None),
 (None,
  3822,
  743,
  382220,
  'NDTV',
  'https://www.ndtv.com/kerala-news',
  743,
  None),
 (None,
  122,
  743,
  560001,
  'The_Hindu',
  'https://www.thehindu.com/news/cities/bangalore/',
  743,
  None),
 (None,
  201,
  743,
  600001,
  'The_Hindu',
  'https://www.thehindu.com/news/cities/chennai/',
  743,
  None),
 (None,
  2013,
  743,
  641001,
  'The_Hindu',
  'https://www.thehindu.com/news/cities/coimbatore/',
 

## Scraping data from News Websites

In [27]:
Nd= {"Member_ID":[],"EMP_ID":[],"DOC_CATEGRY":[],'portalid':[],"Pincode":[],"title":[],"news":[],"date":[]}
for j in range(len(news_data)):
    ## Malappuram News            
    if news_data[j][4]=='TOI':
        req_kerala= requests.get(news_data[j])
        soup_kerala= BeautifulSoup(req_kerala.content)
        dateRegex = re.compile(r"\w{3}\s\d{2}\,\s\d{4}")
        coverpage_news = soup_kerala.find_all('div',{"class":"Mc7GB"})
        link = []
        for i in coverpage_news:
            link.append(i.a['href'])
        for i in link:
            lik = []
            req = requests.get(i)
            blog = BeautifulSoup(req.content)
            for h in blog.findAll('h1'):
                Nd['title'].append(h.text)
            for h in blog.find_all('div',{"class":"_3YYSt"}):
                a=(h.text.strip())
                lik.append(a)
                lik=" ".join(lik)  
                Nd['news'].append(lik)
            Nd['EMP_ID'].append(743)
            Nd['portalid'].append(news_data[j][1])
            Nd['Member_ID'].append(743)
            Nd['Pincode'].append(news_data[j][3])
            Nd['DOC_CATEGRY'].append("News")
  
        for h in soup_kerala.find_all('div',{'class':'Mc7GB'}):
            dateResults = dateRegex.search(h.text).group()
            dateFormatted = datetime.strptime(dateResults, "%b %d, %Y").date()
            Nd['date'].append(dateFormatted)
        
    if news_data[j][4]=='NDTV':
        req_ndtv= requests.get(news_data[j][5])
        soup_ndtv= BeautifulSoup(req_ndtv.content)
        site=[]
        for i in soup_ndtv.findAll('h2',{'class':'newsHdng'}):
            site.append(i.a['href'])
        format= "%B %d, %Y"
        for i in site:
            l=[]
            req= requests.get(i)
            blog= BeautifulSoup(req.content)
            for h in blog.findAll('span',{'class':'pst-by_lnk'}):
                a=(h.text)
                a=a.split()
                a=" ".join(a[1:4])
                d= datetime.strptime(a, format)
                Nd['date'].append(d)
                for h in blog.findAll('h1'):
                    Nd['title'].append(h.text)
                for h in blog.findAll('p'):
                    a=(h.text.strip())
                    l.append(a)
                l=" ".join(l)
                Nd['news'].append(l)
            Nd['EMP_ID'].append(743)
            Nd['portalid'].append(news_data[j][1])
            Nd['Member_ID'].append(743)
            Nd['Pincode'].append(news_data[j][3])
            Nd['DOC_CATEGRY'].append("News")

    if news_data[j][4]=='The_Hindu':
        req_hindu= requests.get(news_data[j][5],allow_redirects=False)
        soup_hin= BeautifulSoup(req_hindu.content)
        s=[]
        for i in soup_hin.findAll('h3',{'class':'title'}):
            s.append(i.a['href'])
        format= "%B %d, %Y"
        for i in s:
            lik=[]
            req= requests.get(i)
            blog= BeautifulSoup(req.content)
            for h in blog.findAll('p',{'class':'publish-time'}):
                a=(h.text)
                a=a.split()
                a=" ".join(a[0:3])
                d= datetime.strptime(a, format)
                Nd['date'].append(d)
                for h in blog.findAll('h1'):
                    Nd['title'].append(h.text)
                for h in blog.findAll('p'):
                    a=(h.text.strip())
                    lik.append(a)
                lik=" ".join(lik)
                Nd['news'].append(lik)
            Nd['EMP_ID'].append(743)
            Nd['portalid'].append(news_data[j][1])
            Nd['Member_ID'].append(743)
            Nd['Pincode'].append(news_data[j][3])
            Nd['DOC_CATEGRY'].append("News")

InvalidSchema: No connection adapters were found for "(None, 6765, 743, 676505, 'TOI', 'https://timesofindia.indiatimes.com/topic/malappuram/', 743, None)"

In [89]:
Ndt= pd.DataFrame.from_dict(Nd)
news=Ndt.values.tolist()

## Deleting Duplicates

In [90]:
a=[]
for i in news:
    if i not in a:
        a.append(i)
a=pd.DataFrame.from_dict(a)
news=a.values.tolist()

## Inserting Data into a Remote Document

In [91]:
remote_db_data="INSERT INTO HT (MEMBER_ID,EMP_ID,DOC_CATEGRY,portalid,Pincode,title,news,date_time) VALUES(%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(remote_db_data, news)
db.commit()

## Upload at Database

In [37]:
db= con.connect(host='localhost', user='root', password='@Amir123', database='college')
cursor= db.cursor()

In [42]:
fetching_news_data= "SELECT * FROM NEWS_SOURCE WHERE VENDOR_ID=743"
cursor.execute(fetching_news_data)
news_data= cursor.fetchall()

In [43]:
news_data

[(None,
  122,
  743,
  122005,
  'NDTV',
  'https://www.ndtv.com/gurgaon-news',
  743,
  None),
 (None,
  201,
  743,
  201005,
  'NDTV',
  'https://www.ndtv.com/ghaziabad-news',
  743,
  None),
 (None,
  2013,
  743,
  201301,
  'NDTV',
  'https://www.ndtv.com/noida-news',
  743,
  None),
 (None,
  2500,
  743,
  250001,
  'NDTV',
  'https://www.ndtv.com/meerut-news',
  743,
  None),
 (None, 282, 743, 282001, 'NDTV', 'https://www.ndtv.com/agra-news', 743, None),
 (None,
  1600,
  743,
  160017,
  'NDTV',
  'https://www.ndtv.com/chandigarh-news',
  743,
  None),
 (None,
  3822,
  743,
  382220,
  'NDTV',
  'https://www.ndtv.com/kerala-news',
  743,
  None),
 (None,
  122,
  743,
  560001,
  'The_Hindu',
  'https://www.thehindu.com/news/cities/bangalore/',
  743,
  None),
 (None,
  201,
  743,
  600001,
  'The_Hindu',
  'https://www.thehindu.com/news/cities/chennai/',
  743,
  None),
 (None,
  2013,
  743,
  641001,
  'The_Hindu',
  'https://www.thehindu.com/news/cities/coimbatore/',
 

## Inserting Data into a Remote Document

In [40]:
remote_db_data="INSERT INTO kf_docmnt (MEMBER_ID,EMP_ID,DOC_CATEGRY,portalid,DOC_TITL,DOC_DESC,DOC_SDATE) VALUES(%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(remote_db_data, news)
db.commit()

InterfaceError: Failed executing the operation; Not enough parameters for the SQL statement

## Malappuram News (Inactive Portal, not in use)

In [19]:
req_MP= requests.get("https://www.onmanorama.com/topic/destination/5/malappuram.html")
soup_MP= BeautifulSoup(req_MP.content)
site=[]
for i in soup_MP.findAll('h2',{'class':'listing-title-002'}):
    site.append('https://www.onmanorama.com'+i.a['href'])

In [21]:
MP= {'MEMBER_ID':[],'EMP_ID':[],'DOC_CATEGRY':[],'portalid':[],'title':[],'news':[],'date':[]}
f= '%B %d, %Y'
for i in site:
    l=[]
    req= requests.get(i)
    blog= BeautifulSoup(req.content)
    for h in blog.findAll('div',{'class':'long-form-text'}):
        a=(h.text)
        #print(a)
        a="".join(a)
        #print(a)
        date= str(blog.findAll('span',{'class':'story-byline-date'}))[116:133]
        MP['date'].append(date)
        for h in blog.findAll('h1'):
            MP['title'].append(h.text)
        for h in blog.findAll('p'):
            a=(h.text.strip())
            l.append(a)
        l=" ".join(l)
        MP['news'].append(l)
        MP['MEMBER_ID'].append("743")
        MP['EMP_ID'].append("743")
        MP['portalid'].append("48")
        MP['DOC_CATEGRY'].append("News")

In [20]:
MP_data= pd.DataFrame(MP)
MP_data

NameError: name 'MP' is not defined

In [54]:
mlp_url= "https://www.onmanorama.com/topic/destination/5/malappuram.html"
portal_id=48
vendor_id=743
location=676504
media_name= 'Kerala News'

mlp_records=[]
a=(1,portal_id,vendor_id,location,media_name,mlp_url)
mlp_records.append(a)
a= pd.DataFrame(mlp_records)
mlp_records= a.values.tolist()

for i in mlp_records:
    mlp_insertion= "INSERT INTO NEWS_SOURCE(NEWS_ID, portal_id,vendor_id,location,media_name,news_url) VALUES (%s,%s,%s,%s,%s,%s)"
    cursor.execute(mlp_insertion, i)
    db.commit()

## Fetching URLs from NEWS_SOURCE

In [55]:
fetching_news_data= "SELECT MEDIA_NAME, NEWS_URL FROM NEWS_SOURCE"
cursor.execute(fetching_news_data)
news_data= cursor.fetchall()

# The Final Code For Nrkindex_Dev Database is given below
## Do not use the abpve code

## Adding URL Source

In [74]:
db= con.connect(host='localhost', user='root', password='@Amir123', database='college')
cursor= db.cursor()

## Fetching data from Source

In [75]:
fetching_news_data= "SELECT * FROM NEWS_SOURCE WHERE VENDOR_ID=743"
cursor.execute(fetching_news_data)
news_data= cursor.fetchall()

## KF_DOCMNT

In [76]:
Nd= {"Member_ID":[],"DOC_CATEGRY":[],"DOC_PUBDATE":[],'portalid':[],"DOC_TITL":[],"DOC_DESC":[],"DOC_SDATE":[]}
for j in range(len(news_data)):
    ## Malappuram News            
    if news_data[j][4]=='TOI':
        req_kerala= requests.get(news_data[j][5])
        soup_kerala= BeautifulSoup(req_kerala.content)
        dateRegex = re.compile(r"\w{3}\s\d{2}\,\s\d{4}")
        coverpage_news = soup_kerala.find_all('div',{"class":"Mc7GB"})
        link = []
        for i in coverpage_news:
            link.append(i.a['href'])
        for i in link:
            lik = []
            req = requests.get(i)
            blog = BeautifulSoup(req.content)
            for h in blog.findAll('h1'):
                Nd['DOC_TITL'].append(h.text)
            for h in blog.find_all('div',{"class":"_3YYSt"}):
                a=(h.text.strip())
                lik.append(a)
                lik=" ".join(lik)  
                Nd['DOC_DESC'].append(lik)
            Nd['portalid'].append(news_data[j][1])
            Nd['Member_ID'].append(743)
            Nd['DOC_CATEGRY'].append("News")
            Nd['DOC_PUBDATE'].append('2023-03-09')
  
        for h in soup_kerala.find_all('div',{'class':'Mc7GB'}):
            dateResults = dateRegex.search(h.text).group()
            dateFormatted = datetime.strptime(dateResults, "%b %d, %Y").date()
            Nd['DOC_SDATE'].append(dateFormatted)
        
    if news_data[j][4]=='NDTV':
        req_ndtv= requests.get(news_data[j][5])
        soup_ndtv= BeautifulSoup(req_ndtv.content)
        site=[]
        for i in soup_ndtv.findAll('h2',{'class':'newsHdng'}):
            site.append(i.a['href'])
        format= "%B %d, %Y"
        for i in site:
            l=[]
            req= requests.get(i)
            blog= BeautifulSoup(req.content)
            for h in blog.findAll('span',{'class':'pst-by_lnk'}):
                a=(h.text)
                a=a.split()
                a=" ".join(a[1:4])
                d= datetime.strptime(a, format)
                Nd['DOC_SDATE'].append(d)
                for h in blog.findAll('h1'):
                    Nd['DOC_TITL'].append(h.text)
                for h in blog.findAll('p'):
                    a=(h.text.strip())
                    l.append(a)
                l=" ".join(l)
                Nd['DOC_DESC'].append(l)
            Nd['portalid'].append(news_data[j][1])
            Nd['Member_ID'].append(743)
            Nd['DOC_CATEGRY'].append("News")
            Nd['DOC_PUBDATE'].append('2023-03-09')

    if news_data[j][4]=='The_Hindu':
        req_hindu= requests.get(news_data[j][5],allow_redirects=False)
        soup_hin= BeautifulSoup(req_hindu.content)
        s=[]
        for i in soup_hin.findAll('h3',{'class':'title'}):
            s.append(i.a['href'])
        format= "%B %d, %Y"
        for i in s:
            lik=[]
            req= requests.get(i)
            blog= BeautifulSoup(req.content)
            for h in blog.findAll('p',{'class':'publish-time'}):
                a=(h.text)
                a=a.split()
                a=" ".join(a[0:3])
                d= datetime.strptime(a, format)
                Nd['DOC_SDATE'].append(d)
                for h in blog.findAll('h1'):
                    Nd['DOC_TITL'].append(h.text)
                for h in blog.findAll('p'):
                    a=(h.text.strip())
                    lik.append(a)
                lik=" ".join(lik)
                Nd['DOC_DESC'].append(lik)
           
            Nd['portalid'].append(news_data[j][1])
            Nd['Member_ID'].append(743)
            Nd['DOC_CATEGRY'].append("News")
            Nd['DOC_PUBDATE'].append('2023-03-09')

In [77]:
Ndt= pd.DataFrame.from_dict(Nd)
news=Ndt.values.tolist()

In [78]:
len(Nd['portalid'])

147

## Delete Duplicates

In [79]:
a=[]
for i in news:
    if i not in a:
        a.append(i)
a=pd.DataFrame.from_dict(a)
news=a.values.tolist()

## Insert into KF_Docmnt

In [80]:
remote_db_data="INSERT INTO kf_docmnt (MEMBER_ID,DOC_CATEGRY,DOC_PUBDATE,portalid,DOC_TITL,DOC_DESC,DOC_SDATE) VALUES(%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(remote_db_data, news)
db.commit()

In [81]:
news

[[743,
  'News',
  '2023-03-09',
  122,
  ' Gurugram Civil Engineer Shot Over Parking Row, Case Filed Against 10: Cops ',
  'The incident happened on Friday, said the police. (Representative) A civil engineer was allegedly shot by over 10 unidentified people over a car parking issue in Sohna area here, police said on Saturday. The incident happened on Friday, they said. The victim Gautam Khatana, in his complaint, said that as he came out of a cafe, more than 10 people in an inebriated condition approached him and clashed with him over a car parking issue. "Two of them were carrying weapons in their hand and started abusing me and asked why I parked my car there because the parking space was only for their friends," the complainant said. Following this, the group shot at Khatana, ransacked the cafe and also damaged his car, the engineer said in his complaint. After getting information, police reached the spot and recovered two bullet shells from there while the group managed to flee aw